In [ ]:
!pip install fastavro

In [1]:
from fastavro import writer, reader, parse_schema

In [2]:
schema = {
    'doc': 'A weather reading.',
    'name': 'Weather',
    'namespace': 'test',
    'type': 'record',
    'fields': [
        {'name': 'station', 'type': 'string'},
        {'name': 'time', 'type': 'long'},
        {'name': 'temp', 'type': 'int'},
    ],
}
parsed_schema = parse_schema(schema)

# 'records' can be an iterable (including generator)
records = [
    {u'station': u'011990-99999', u'temp': 0, u'time': 1433269388},
    {u'station': u'011990-99999', u'temp': 22, u'time': 1433270389},
    {u'station': u'011990-99999', u'temp': -11, u'time': 1433273379},
    {u'station': u'012650-99999', u'temp': 111, u'time': 1433275478},
]

In [ ]:
# Writing
with open('weather.avro', 'wb') as out:
    writer(out, parsed_schema, records)

In [3]:
# Reading
with open('weather.avro', 'rb') as fo:
    for record in reader(fo):
        print(record)

{'station': '011990-99999', 'time': 1433269388, 'temp': 0}
{'station': '011990-99999', 'time': 1433270389, 'temp': 22}
{'station': '011990-99999', 'time': 1433273379, 'temp': -11}
{'station': '012650-99999', 'time': 1433275478, 'temp': 111}


In [4]:
import pandas as pd

In [5]:
# Writie to a Pandas data frame
with open('weather.avro', 'rb') as fo:
    df = pd.DataFrame([record for record in reader(fo)])   

In [6]:
df

,station,time,temp
0,011990-99999,1433269388,0
1,011990-99999,1433270389,22
2,011990-99999,1433273379,-11
3,012650-99999,1433275478,111


In [ ]:
df.columns

# AWS Setup

In [7]:
import boto3
s3 = boto3.resource('s3')

In [8]:
for bucket in s3.buckets.all():
    print(bucket.name)

moti-poc-buucker-xyz1


In [ ]:
bucket_name = "s3://moti-poc-buucker-xyz1/input/"
a = bucket_name+"weather.avro"
a

In [ ]:
# Read from AWS S3 bucket

# Writie to a Pandas data frame
bucket_name = f"s3://moti-poc-buucker-xyz1/input/"
file_name = bucket_name+"weather.avro"
print(file_name)
with open(file_name, 'rb') as fo:
    df_1 = pd.DataFrame([record for record in reader(fo)])   

In [ ]:
df_1

In [ ]:
import io

In [ ]:
s3client = boto3.client('s3')
bucket_name="security-price-12345"
s3_file = "securitydata.json"

s3client.get_object(Bucket=bucket_name, Key=s3_file)

In [ ]:
!aws s3api get-bucket-notification --bucket moti-poc-buucker-xyz
s3://moti-poc-buucker-xyz/input/

In [ ]:
s3client = boto3.client('s3')
bucket_name="moti-poc-buucker-xyz"
s3_file = "input/weather.avro"

obj = s3client.get_object(Bucket=bucket_name, Key=s3_file)
obj

In [ ]:
io.BytesIO(obj['Body'].read())

In [ ]:
obj = s3client.get_object(Bucket=bucket_name, Key=s3_file)

In [ ]:
a = io.BytesIO(obj['Body'].read())

In [ ]:
s3client = boto3.client('s3')
bucket_name="security-price-12345"
s3_file = "securitydata.json"

obj = s3client.get_object(Bucket=bucket_name, Key=s3_file)
obj

In [ ]:
a = io.BytesIO(obj['Body'].read())
a

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('security-price-12345')

for obj in bucket.objects.all():
    key = obj.key
    body = obj.get()['Body'].read()

In [ ]:
s3client = boto3.client('s3')
bucket_name="security-price-12345"
s3_file = "securitydata.json"

obj = s3client.get_object(Bucket=bucket_name, Key=s3_file)
print(obj['Body'].read().decode('utf-8'))

In [10]:
import boto3
s3client = boto3.client('s3')
bucket_name="moti-poc-buucker-xyz1"
s3_file = "input/weather.avro"

avro_file = s3client.get_object(Bucket=bucket_name, Key=s3_file)['Body']

In [11]:
import io
avro_file_buffer = io.BytesIO(avro_file.read())

In [12]:
#Configure Avro Reader
reader = reader(avro_file_buffer)

In [13]:
# Load records in memory
records = [r for r in reader]

In [14]:
import pandas as pd
df = pd.DataFrame.from_records(records)
df

,station,time,temp
0,011990-99999,1433269388,0
1,011990-99999,1433270389,22
2,011990-99999,1433273379,-11
3,012650-99999,1433275478,111


In [20]:
bucket_name="moti-poc-buucker-xyz1"
s3_out_file = "output/weather.csv"
s3_filename = "s3://"+bucket_name +"/" +s3_out_file
s3_filename

's3://moti-poc-buucker-xyz1/output/weather.csv'

In [27]:
bucket = "moti-poc-buucker-xyz1"
key = "output/weather.csv"

# Create buffer
csv_buffer =  io.StringIO()

# Write dataframe to buffer
df.to_csv(csv_buffer, sep="|", index=False)

# Create S3 object
s3_client = boto3.client("s3")

# Write buffer to S3 object
#s3_resource.Object()

 # write stream to S3
obj = s3_client.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
    